In [1]:
# Import required library

In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn import metrics, linear_model
from sklearn.model_selection import GridSearchCV, train_test_split

In [3]:
# Question 1

In [4]:
# Q1 Load files

In [5]:
with open('/Users/ryanhuang/Desktop/107-2/statistical learning/Assignments/Assign3/phone_train.pickle', 'rb') as fh1:
    traindata = pickle.load(fh1)

with open('/Users/ryanhuang/Desktop/107-2/statistical learning/Assignments/Assign3/phone_test1.pickle', 'rb') as fh2:
    test = pickle.load(fh2)

x_train = traindata.iloc[:,0:3]
y_train = traindata.iloc[:,3]

x_test = test.iloc[:,0:3]
y_test = test.iloc[:,3]

In [6]:
# Self-defined generative classification class

In [7]:
class mypgc:
    def fit(self, X_train, Y_train):
        def model_info(x_train,y_train, ylab):
            info_collection = dict()
            for lab in ylab:
                info = dict()
                data = x_train.loc[y_train == lab,]
                info['mu'] = data.mean(0)
                info['cov'] = data.cov(0)
                info['prec'] = np.linalg.inv(data.cov(0))
                info['detcov'] = np.linalg.det(data.cov(0))
                info['n'] = data.shape[0]
                info['prior'] = data.shape[0] / x_train.shape[0]
                info_collection[lab] = info
            return info_collection
        self.xtrain = X_train
        self.ytrain = Y_train
        self.ylab = self.ytrain.unique()
        self.model = model_info(self.xtrain,self.ytrain, self.ylab)

    def predict(self, x_test):
        prediction = []
        for i in range(x_test.shape[0]):
            p_list = []
            for lab in self.ylab:
                p_list.append(((2 * np.pi) ** (-3 / 2)) \
                              * (self.model[lab]["detcov"] ** (-1 / 2)) \
                              * np.exp(
                                        (-0.5) * (x_test.iloc[i,:] - self.model[lab]["mu"]).T \
                                        .dot(self.model[lab]["prec"]) \
                                        .dot(x_test.iloc[i,:] - self.model[lab]["mu"])) \
                              * self.model[lab]["prior"])
            prediction.append(self.ylab[np.asarray(p_list).argmax()])
        return np.asarray(prediction)

In [8]:
# Model training

In [9]:
pgc1 = mypgc()
pgc1.fit(x_train, y_train)
pred = pgc1.predict(x_test.iloc[0:20,])
print(pgc1.model)

{'Phoneonback': {'mu': x    0.206933
y   -0.026152
z    9.796376
dtype: float64, 'cov':           x         y         z
x  0.002990 -0.001958  0.002848
y -0.001958  0.002299 -0.002425
z  0.002848 -0.002425  0.004888, 'prec': array([[ 943.63669582,  469.54782044, -316.78479596],
       [ 469.54782044, 1146.63145078,  295.42598486],
       [-316.78479596,  295.42598486,  535.76812508]]), 'detcov': 5.673178805678628e-09, 'n': 28566, 'prior': 0.17095459523510295}, 'Phoneonbottom': {'mu': x    0.190768
y    9.785847
z    0.146557
dtype: float64, 'cov':           x         y         z
x  0.002739 -0.004438  0.001520
y -0.004438  0.010268 -0.003281
z  0.001520 -0.003281  0.002480, 'prec': array([[1229.24902343,  503.42022589,  -87.22678994],
       [ 503.42022589,  374.93550309,  187.61601574],
       [ -87.22678994,  187.61601574,  705.02302834]]), 'detcov': 1.1952712798255282e-08, 'n': 27842, 'prior': 0.1666217825574367}, 'Phoneonfront': {'mu': x    0.112704
y    0.142651
z   -9.735796
dtyp

In [10]:
# Model result

In [11]:
metrics.accuracy_score(y_test.iloc[0:20,], pred)

1.0

In [12]:
# Question 2

In [13]:
# Q_2.1 Data cleaning

In [14]:
# read the data, N/A values are '?'
header_list = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation',
               'relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country', 'attribute']
q2_train_path = '/Users/ryanhuang/Desktop/107-2/statistical learning/Assignments/Assign3/adult.data.txt'
q2_train_raw = pd.read_csv(q2_train_path, header = None, names = header_list, na_values=['?'], skipinitialspace=True)
q2_test_path = '/Users/ryanhuang/Desktop/107-2/statistical learning/Assignments/Assign3/adult.test.txt'
q2_test_raw = pd.read_csv(q2_test_path, header = None, names = header_list, na_values=['?'], skipinitialspace=True)
q2_train_raw.dropna(axis=0, inplace=True)
q2_test_raw.dropna(axis=0, inplace=True)
q2_test_raw['age'] = q2_test_raw['age'].astype(int)

In [15]:
# Clean the category variables with 1 hot encoding
class_var_list = ['workclass', 'education', 'marital-status','occupation','relationship','race','native-country','sex']
class_others = []
small_class_col = []
for col in class_var_list:
    other_value = q2_train_raw[col].value_counts()\
                         .index\
                         [q2_train_raw[col].value_counts()<10]
    if len(other_value) > 0:
        class_others.append(other_value[:len(other_value)])
        small_class_col.append(col)
for col in small_class_col:
    q2_train_raw.loc[:,col] = q2_train_raw.loc[:,col].apply(lambda x: 'others' if x in class_others else x)
    q2_test_raw.loc[:,col] = q2_test_raw.loc[:,col].apply(lambda x: 'others' if x in class_others else x)

In [16]:
# 1-hot encoding for class_vars
# The conti. vars are from column_1 to column_7, column_8 is the attribute
q2_train = pd.get_dummies(q2_train_raw)
q2_test  = pd.get_dummies(q2_test_raw)
q2_train_raw['occupation'].value_counts()
q2_test_raw['occupation'].value_counts()

Exec-managerial      1992
Craft-repair         1990
Prof-specialty       1970
Sales                1824
Adm-clerical         1819
Other-service        1596
Machine-op-inspct    1004
Transport-moving      744
Handlers-cleaners     696
Tech-support          508
Farming-fishing       491
Protective-serv       332
Priv-house-serv        89
others                  5
Name: occupation, dtype: int64

In [17]:
# check different columns in test and training
train_col = q2_train.columns.tolist()
test_col = q2_test.columns.tolist()
print(list(set(train_col).difference(set(test_col)))) # in train not in test
print(list(set(test_col).difference(set(train_col)))) # in test not in train

['attribute_>50K', 'native-country_others', 'attribute_<=50K']
['attribute_>50K.', 'attribute_<=50K.']


In [18]:
# add native-country_others to testing data
q2_test['native-country_others'] = 0

# rename columns
q2_test = q2_test.rename({'attribute_<=50K.':'attribute_<=50K','attribute_>50K.':'attribute_>50K'}, axis='columns')

# Rearrange the col order
q2_test = q2_test[q2_train.columns]

In [19]:
# seperate train_x, train_y, test_x, test_y
x_train, y_train = np.asarray(q2_train.iloc[:,:-2]), np.asarray(q2_train.iloc[:,-1])
x_test, y_test   = np.asarray(q2_test.iloc[:,:-2]), np.asarray(q2_test.iloc[:,-1])

# x_train
q2_train.iloc[:,:-2].describe()
# y_train
q2_train.iloc[:,-1].describe()
# x_test
q2_test.iloc[:,:-2].describe()
# y_test
q2_test.iloc[:,-1].describe()

count    15060.000000
mean         0.245684
std          0.430506
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: attribute_>50K, dtype: float64

In [20]:
# Q_2.2: Derive the gradient and hession matrix for the new E(w).
# Gradient = Λw + ∑(yn - tn)xn
# Hession = Λ + x^T * R * x, where R is NxN diagonal matrix with the diagonal is yn * ( 1-yn )

In [21]:
# Q_2.3: Self-define Logistics Regresser

In [18]:
class mylogistic_l2:
    def __init__(self, reg_vec, max_iter, tol, add_intercept = True):
        self.reg_vec = reg_vec
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept

    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, x_train, y_train):
        if self.add_intercept:
            self.x_train = np.c_[ x_train, np.ones(x_train.shape[0]) ]
        else:
            self.x_train = x_train
        self.y_train = y_train

        # initial w vector given by ridge regression
        w = np.linalg.inv( self.x_train.T.dot(self.x_train) +
                                (np.diag(self.reg_vec).mean()
                                    * np.identity(self.x_train.shape[1]))) \
                .dot(self.x_train.T) \
                .dot(self.y_train)

        # IRLS Loop to find optimal w
        error = []
        best_w = None
        for i in range(self.max_iter):
            y = self.__sigmoid(np.dot(self.x_train, w)) # y = n(data size) * 1, each element is in [0,1]
            # y = 1 / (1 + np.exp((-self.w).T.dot(self.x_train.T)))
            t = self.y_train  # t = n(data size) * 1
            gradient = self.reg_vec.dot(w) + self.x_train.T.dot(y - t)  # gradient = m(feature size) * 1
            R = np.diagflat(y * (1 - y))
            hession = self.reg_vec + self.x_train.T.dot(R).dot(self.x_train)
            w = w - np.linalg.inv(hession).dot(gradient)
            new_error = 0.5 * w.T.dot(self.reg_vec).dot(w) - (t * np.log(y) + (1 - t * np.log((1 - y)))).sum()
            if i > 0 and new_error < error[-1]:
                best_w = w
            if i > 0 and abs(new_error - error[-1]) <= self.tol:
                break
            else:
                error.append(new_error)

            # w is determined after the IRLS converged
            self.w = best_w
        self.error_list = error

    def predict(self, x_test):
        if self.add_intercept:
            self.x_test = np.c_[x_test, np.ones(x_test.shape[0])]
        else:
            self.x_test = x_test
        pred = self.__sigmoid(np.dot(self.x_test, self.w))
        return pred.round()

In [19]:
# case_1: lambda = 1 for all coefficients
lambda_vec_1 = np.diagflat([1] * (x_train.shape[1]+1))
logic1 = mylogistic_l2(reg_vec = lambda_vec_1, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic1.fit(x_train, y_train)
ypred1 = logic1.predict(x_test)
print('accuracy is', metrics.accuracy_score(y_test, ypred1))
#print('coefficients w are', logic1.w)

accuracy is 0.84800796812749


In [21]:
#print("    num of iteration =", logic1.niter)
print("    obj_function value=", logic1.error_list)
print("    w[0:5] (numerical-valued coef) = ", logic1.w[0:5])
print("    w[10:15] (binary-valued coef) = ", logic1.w[10:15])
print("    w[-1] (incercept) = ", logic1.w[-1])

    obj_function value= [-34112.21628423518, -31313.63812950806, -33976.702408429876, -37169.1168387285, -38992.06826924892, -39368.33721684105, -39382.786108549466, -39382.80024352845]
    w[0:5] (numerical-valued coef) =  [2.48542874e-02 7.26628938e-07 1.86052927e-01 3.16575978e-04
 6.38741410e-04]
    w[10:15] (binary-valued coef) =  [-0.7932101  -0.61782768 -1.17218718 -0.45123745 -0.54415846]
    w[-1] (incercept) =  -3.32567134748659


In [25]:
#case_2: lambda = 1 for all but the intercept, no regularization for intercept term.
lambda_vec_2 = np.diagflat([1] * x_train.shape[1] + [0])
logic2 = mylogistic_l2(reg_vec = lambda_vec_2, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic2.fit(x_train, y_train)
ypred2 = logic2.predict(x_test)
print('accuracy is', metrics.accuracy_score(y_test, ypred2))
print('coefficients w are', logic2.w)

accuracy is 0.847875166002656
coefficients w are [ 2.54295548e-02  7.50945179e-07  2.95317307e-01  3.17014683e-04
  6.39715093e-04  2.94903835e-02  7.06908026e-01  1.75726588e-02
  2.08885351e-01  3.82528927e-01 -2.80056412e-01 -1.04778523e-01
 -9.31060027e-01  9.06214969e-02 -1.06411979e-01 -5.47663220e-02
  7.05949201e-01  5.35843404e-01  1.16205771e-01  1.37324858e-01
 -4.02159115e-01 -1.13538481e-01 -7.45191900e-02  6.78820737e-02
 -1.97258204e-02 -1.15573834e-02 -1.15970847e+00  2.66928077e-01
  2.16318781e-02 -5.26718255e-01  1.61472912e+00  1.36800663e+00
 -4.92649813e-01 -1.01532182e+00 -6.05953641e-01 -3.42092226e-01
  1.49997638e-01  2.14446830e-01  9.50862583e-01 -8.31719218e-01
 -5.34580247e-01 -1.13006170e-01 -6.63135450e-01 -1.56103460e+00
  6.64398431e-01  7.37162512e-01  4.41486031e-01  8.04743727e-01
  5.92587204e-02 -3.18880788e-01 -4.27131102e-02  1.99548659e-01
 -5.82604954e-01 -9.37093672e-01  7.54141887e-02  1.28744889e+00
 -3.71026282e-01  3.94087782e-01  4.28275

In [33]:
#case_3: lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for incercept
lambda_vec_3 = np.diagflat([1] * 7 + [0.5] *(x_train.shape[1]-7) + [0])
logic3 = mylogistic_l2(reg_vec = lambda_vec_3, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic3.fit(x_train, y_train)
ypred3 = logic3.predict(x_test)
metrics.accuracy_score(y_test, ypred3)
print('accuracy is', metrics.accuracy_score(y_test, ypred3))
print('coefficients w are', logic3.w)

accuracy is 0.847675962815405
coefficients w are [ 2.54695730e-02  7.52271368e-07  3.19083151e-01  3.17302273e-04
  6.40217484e-04  2.95117222e-02  6.64828492e-01 -2.52083943e-02
  1.66636693e-01  3.41825852e-01 -3.22314402e-01 -1.47691017e-01
 -1.34290572e+00  2.18750528e-01 -2.51809332e-03  2.46020712e-02
  9.74746677e-01  7.53811068e-01  2.92003188e-01  2.92813679e-01
 -4.19830245e-01 -1.04556378e-01 -1.13126008e-01 -3.79911613e-02
  3.74637445e-02 -7.29987343e-02 -2.08397342e+00  1.86159291e-01
  5.46437935e-02 -5.72264656e-01  1.82621717e+00  1.39690722e+00
 -5.47321824e-01 -1.05939403e+00 -6.55939889e-01 -3.88203989e-01
  2.00468981e-01  2.64389652e-01  1.00237908e+00 -7.88410386e-01
 -4.89003477e-01 -6.26537026e-02 -6.18079609e-01 -2.02574798e+00
  7.15127317e-01  7.91547668e-01  4.92474266e-01  8.59371725e-01
  1.09407405e-01 -4.51270945e-01 -8.41153035e-02  2.32836754e-01
 -5.91522369e-01 -9.22950452e-01  1.11386339e-01  1.25436503e+00
 -3.82279092e-01  4.12693607e-01  4.11262

In [ ]:
# Q_2.4: Parameter tuning

In [38]:
def lambda_vec(a1, a2):
    return np.diagflat([a1] * 7 + [a2] * 97 + [0])

x_subtrain, x_tuning, y_subtrain, y_tuning = train_test_split(x_train,y_train, train_size=0.9)
lambda_list = np.logspace(-2,2,10)

/Users/ryanhuang/.conda/envs/untitled/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [39]:
# stage_1
stage_1 = []
for i in lambda_list:
    q2_4_logic = mylogistic_l2(max_iter = 1000, tol = 1e-5, add_intercept = True, reg_vec=lambda_vec(i,i))
    q2_4_logic.fit(x_subtrain, y_subtrain)
    ypred = q2_4_logic.predict(x_tuning)
    stage_1.append(metrics.accuracy_score(y_tuning, ypred))

print('a1, a2 are', lambda_list[stage_1.index(max(stage_1))])

a1, a2 are 1.6681005372000592


In [40]:
# stage 2: fixed a1

stage_2 = []
for i in lambda_list:
    q2_4_logic = mylogistic_l2(max_iter = 1000, tol = 1e-5, add_intercept = True, reg_vec=lambda_vec(lambda_list[stage_1.index(max(stage_1))],i))
    q2_4_logic.fit(x_subtrain, y_subtrain)
    ypred = q2_4_logic.predict(x_tuning)
    stage_2.append(metrics.accuracy_score(y_tuning, ypred))
print('new a2 is', lambda_list[stage_2.index(max(stage_2))])

new a2 is 4.6415888336127775


In [41]:
# stage 3: fixed a2
stage_3 = []
for i in lambda_list:
    q2_4_logic = mylogistic_l2(max_iter = 1000, tol = 1e-5, add_intercept = True, reg_vec=lambda_vec(i,lambda_list[stage_2.index(max(stage_2))]))
    q2_4_logic.fit(x_subtrain, y_subtrain)
    ypred = q2_4_logic.predict(x_tuning)
    stage_3.append(metrics.accuracy_score(y_tuning, ypred))
print('new a1 is', lambda_list[stage_3.index(max(stage_3))])

new a1 is 0.01


In [42]:
# Final summary
q2_4_logic_final = mylogistic_l2(max_iter = 1000, tol = 1e-5, add_intercept = True, reg_vec=lambda_vec(lambda_list[stage_3.index(max(stage_3))],lambda_list[stage_2.index(max(stage_2))]))
q2_4_logic_final.fit(x_train, y_train)
q2_5_logic_final_pred = q2_4_logic_final.predict(x_test)
print('The best pair of (a1, a2) is (',lambda_list[stage_3.index(max(stage_3))],',',lambda_list[stage_2.index(max(stage_2))],')')
print('The resulting accuracy is', metrics.accuracy_score(y_test, q2_5_logic_final_pred))

The best pair of (a1, a2) is ( 0.01 , 4.6415888336127775 )
The resulting accuracy is 0.84867197875166


In [43]:
print('The best pair of (a1, a2) is (',lambda_list[stage_3.index(max(stage_3))],',',lambda_list[stage_2.index(max(stage_2))],')')


The best pair of (a1, a2) is ( 0.01 , 4.6415888336127775 )


In [29]:
# Q_2.5: Compare with standard logistics regression in sklearn library
grid={"C":np.logspace(-2,2,10),"penalty":["l2"]}
q2_5_logic = linear_model.LogisticRegression(solver='lbfgs', max_iter=1000)
logreg_cv=GridSearchCV(q2_5_logic,grid)
logreg_cv.fit(x_subtrain,y_subtrain)
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)
q2_5_logic_final = linear_model.LogisticRegression(C=logreg_cv.best_params_['C'], penalty='l2')
q2_5_logic_final.fit(x_train, y_train)

/Users/ryanhuang/.conda/envs/untitled/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


tuned hpyerparameters :(best parameters)  {'C': 0.01, 'penalty': 'l2'}
accuracy : 0.7906428439860012


/Users/ryanhuang/.conda/envs/untitled/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [44]:
# Compared with the standard model, our own model performances better with 5% higher accuracy score
# However, when it comes to efficiency, the standard model is so much faster
# Comparing the coefficients:
print('standard logistc:', q2_5_logic_final.coef_)
print('self-defined logistics:', q2_4_logic_final.w)

standard logistc: [[-4.43024900e-03 -4.44696056e-06 -1.07673189e-03  3.29268998e-04
   7.23501864e-04 -5.30129838e-03  1.11704532e-05 -2.71400492e-06
  -2.62358318e-04  3.01794247e-05 -1.68307432e-05 -7.26456508e-06
  -6.66149762e-07 -2.59136978e-05 -3.63494994e-05 -1.17330099e-05
  -4.37246854e-06 -8.82781214e-06 -1.93126219e-05 -1.46962108e-05
  -6.23255362e-06 -9.48487044e-06  7.71833849e-05  2.14917729e-05
  -1.90942955e-04  5.24045572e-05 -1.75367020e-06  2.61823153e-05
  -9.61265636e-05 -1.25696481e-04  6.53782706e-07  2.97104430e-04
  -1.14511025e-05 -3.52568836e-04 -2.95307373e-05 -2.69949587e-05
  -8.80763188e-05 -4.18075075e-05  9.19237969e-05 -3.12678338e-05
  -4.34340521e-05 -4.63445175e-05 -1.18851681e-04 -5.80320458e-06
   6.95120443e-05  4.97606820e-06 -1.86941032e-05  1.34495820e-06
  -2.17704683e-05 -1.91083596e-07  2.65146064e-04 -2.28625709e-04
  -3.12728230e-05 -1.80902915e-04 -1.09523799e-04  3.66952797e-05
  -1.13932894e-05 -7.34624828e-06 -5.40734221e-05 -6.67688